# Chapter 4 - Trees and Graphs
Marla Odell

In [3]:
import sys
from trees_and_nodes import *
from graph import *
from linked_list import *
from stack_and_queue import *

**(4.1) Route Between Notes:** Given a directed graph, design an algorithm to find out whether there is a route between two nodes.

In [4]:
def route_between_nodes(g, v, w):
    if v == w:
        return True
    for x in g.vertices.values():
        x.state = State.UNSEEN
    q = Queue()
    v.state = State.LOOKING_AT
    q.enqueue(v)
    while len(q) > 0:
        x = q.dequeue()
        for adj_vertex in x.adj:
            if adj_vertex.state == State.UNSEEN:
                if adj_vertex == w:
                    return True
                adj_vertex.state = State.LOOKING_AT
                q.enqueue(adj_vertex)
        x.state = State.SEEN

**(4.2) Minimal Tree:** Given a sorted (increasing order) array with unique integer elements, write an algorithm to create a binary search tree with minimal height.

In [5]:
class MinimalTree(BinarySearchTree):
    
    def __init__(self, sorted_array = None):
        self.root = self.put_sorted_array(sorted_array)
        
    def put_sorted_array(self, sorted_array):
        if sorted_array:
            self.root = self._put_sorted_array(sorted_array)    
            return self.root
        return None
    
    def _put_sorted_array(self, sorted_array):
        if not sorted_array:
            return None
        middle = len(sorted_array) // 2
        root = BinaryTreeNode(sorted_array[middle])
        root.left = self._put_sorted_array(sorted_array[:middle])
        root.right = self._put_sorted_array(sorted_array[middle + 1:])
        return root
    
    def _max_depth(self, node): 
        if node is None: 
            return 0 
        return max(self._max_depth(node.left) + 1, self._max_depth(node.right) + 1)    

    def max_depth(self):
        return self._max_depth(self.root)

**(4.3) List of Depths:** Given a binary tree, design an algorithm which creates a linked list of all the nodes at each depth (e.g., if you have a tree with depth D, you'll have D linked lists).

In [6]:
def list_of_depths(root):
    previous, current = None, 0
    q = Queue()
    q.enqueue((root, current))
    depths = []
    while len(q) > 0:
        x, current = q.dequeue()
        if current == previous:
            ll = depths.pop()
        else:
            ll = LinkedList()
        ll.add_back(x)
        depths.append(ll)
        if x.left:
            q.enqueue((x.left, current + 1)) #Add to the start
        if x.right:
            q.enqueue((x.right, current + 1)) #Add to the start
        previous = current
    return depths

def string_representation(depths): #Used in testing and debugging
    string = ""
    for d in depths:
        string += str(d) + "|"
    print(string[:-1])

In [7]:
def list_of_depths(bt):
    prior = None
    current = 0
    q = Queue()
    q.enqueue((bt.root, current))
    depths = []
    while len(q) != 0:
        x, current = q.dequeue()
        if current != prior:
            ll = LinkedList()
        else:
            ll = depths.pop()
        ll.add_back(x)
        depths.append(ll)
        if x.left:
            q.enqueue((x.left, current+1))
        if x.right:
            q.enqueue((x.right, current+1))
        prior = current
    return depths

def string_builder(depths):
    string = ""
    for d in depths:
        string += str(d) + " | "
    return string[:-3]

**(4.4) Check Balanced:** Implement a function to check if a binary tree is balanced. For the purposes of this question, a balanced tree is defined to be a tree such that the heights of the two subtrees of any node never differ by more than one.

In [8]:
#Uses mas_depth(node) from (4.2)

def check_balanced(bst):
    root = bst.root
    if not root: 
        return True
    elif abs(max_depth_from_node(bst, root.left) - max_depth_from_node(bst, root.right)) <= 1: #Check if balanced
        return True
    return False

def max_depth_from_node(bst, node): 
    if node is None: 
        return 0 
    return max(max_depth_from_node(bst, node.left) + 1, max_depth_from_node(bst, node.right) + 1)    

**(4.5) Validate BST:** Implement a function to check if a binary tree is a binary search tree.

In [9]:
def validate_bst(tree):

    def _validate_bst(node, left_bound, right_bound):
        if not node:
            return True
        return node.val >= left_bound and node.val <= right_bound and \
            _validate_bst(node.left, left_bound, node.val) and \
            _validate_bst(node.right, node.val, right_bound)
    
    node = tree.root
    return _validate_bst(tree.root, -sys.maxsize - 1, sys.maxsize)

**(4.6) Successor:** Write an algorithm to find the "next" node (i.e., in-order successor) of a given node in a binary search tree. You may assume that each node has a link to its parent.

In [10]:
def successor(bst, node):
    
    def _successor(root, node): 
        if node.right: #Successor in right subtree
            node = node.right
            while node: 
                if not node.left: 
                    return node
                node = node.left 
        above = node.parent #Travel up
        while above: 
            if node != above.right : 
                return above 
            node = above 
            above = above.parent 
            
    return _successor(bst.root, node)

**(4.7) Build Order:** You are given a list of projects and a list of dependencies (which is a list of pairs of projects, where the second project is dependent on the first project). All of a project's dependencies must be built before the project is. Find a build order that will allow the projects to be built. If there is no valid build order, return an error.

In [11]:
def build_order(projects, dependencies):
    
    def _dfs(order, v):
        if v.state == State.LOOKING_AT:
            return False
        if v.state == State.UNSEEN:
            v.state = State.LOOKING_AT
            for adj in v.adj.keys(): #Loop through edges
                if not _dfs(order, adj):
                    return False
            v.state = State.SEEN #Mark as visited
            order.insert(0, v.key)
        return True
    
    g = build_graph(projects, dependencies)
    order = []
    for v in g.vertices.values():
        if not _dfs(order, v):
            return None
    return order

**(4.8) First Common Ancestor:** Design an algorithm and write code to find the first common ancestor of two nodes in a binary tree. Avoid storing additional nodes in a data structure. NOTE: This is not necessarily a binary search tree.

In [12]:
def first_common_ancestor(tree, node_1, node_2):

    def _first_common_ancestor(root, node_1, node_2): # pass root in
        if not root or not node_1 or not node_2:
            return None
        if root == node_1 or root == node_2:
            return root
        left_side = _first_common_ancestor(root.left, node_1, node_2)
        right_side = _first_common_ancestor(root.right, node_1, node_2)
        if left_side and right_side:
            return root
        if left_side:
            return left_side
        else:
            return right_side

    return _first_common_ancestor(tree.root, node_1, node_2)

**(4.9) BST Sequence:** A binary search tree was created by traversing through an array from left to right and inserting each element. Given a binary search tree with distinct elements, print all possible arrays that could have led to this tree.

In [13]:
def bst_sequences(bst):
  
    def _bst_sequences_partial(partial, root):
        if len(root) == 0:
            return [partial]
        list_of_seq = []
        for i, node in enumerate(root):
            next_subtree = root[:i] + root[i + 1:]
            if node.left:
                next_subtree.append(node.left)
            if node.right:
                next_subtree.append(node.right)
            possible_seq = _bst_sequences_partial(partial + [node.val], next_subtree) #Recurse on remaining
            list_of_seq += possible_seq
        return list_of_seq
    
    return _bst_sequences_partial([], [bst.root])

**(4.10) Check Subtree:** T1 and T2 are two very large binary trees, with TI much bigger than T2. Create an algorithm to determine if T2 is a subtree o f Tl. 

A tree T2 is a subtree of T1 if there exists a node n in T1 such that the subtree of n is identical to T2. That is, if you cut off the tree at node n, the two trees would be identical,

In [14]:
def check_subtree(T1, T2): #Check if T2 is a subtree of T1
    
    def _same_tree(T1_node, T2_node): #Check if two trees are the same
        if not T2_node: 
            return True
        elif not T1_node: 
            return False 
        elif T1_node == T2_node:
            left_side = _same_tree(T1_node.left , T2_node.left)
            right_side = _same_tree(T1_node.right, T2_node.right) 
            return left_side and right_side
        return False
        
    def _check_subtree(T1_node, T2_node):
        if not T1_node:
            return False
        elif T1_node == T2_node:
            return _same_tree(T1_node, T2_node)
        return _check_subtree(T1_node.left, T2_node) and _check_subtree(T1_node.right, T2_node)

    if not T1.root:
        return False
    elif not T2.root: 
        return True
    return _check_subtree(T1.root, T2.root)

**(4.11) Random Node:** You are implementing a binary tree that has a method ```getRandomNode()``` which returns a random node from the tree. All nodes should be equally likely to be chosen. Design and implement an algorithm for get RandomNode, and explain how you would implement the rest of the methods.

In [15]:
class RandomBinaryTree(BinarySearchTree):
            
    def getRandomNode(self, seed = None):
        if seed:
            return get_numbered_node(self.root, seed)
        total_nodes = count_subtree_nodes(self.root)
        random_number = random.randint(0, total_nodes - 1) #Generate a random number
        return get_numbered_node(self.root, random_number)

def count_subtree_nodes(root):
    count = 0
    if root:
        count += count_subtree_nodes(root.left) \
              + count_subtree_nodes(root.right) \
              + 1 #Recusively count the subtrees
    return count
    
def get_numbered_node(root, number):
    if number == 0: #Found the n-th node
        return root
    if root.left: #Continue to traverse and decrement n
        if number - 1 < count_subtree_nodes(root.left):
            return get_numbered_node(root.left, number - 1)
        elif node.right:
            return get_numbered_node(root.right, number - 1 - root.left.count)
    if root.right:
        return get_numbered_node(root.right, number - 1)
    return None

**(4.12) Paths with Sum:** You are given a binary tree in which each node contains an integer value (which might be positive or negative). Design an algorithm to count the number of paths that sum to a given value. The path does not need to start or end at the root or a leaf, but it must go downwards (traveling only from parent nodes to child nodes).

In [16]:
def paths_with_sum(binary_tree, target_sum):
    
    def _paths_with_sum(node, goal, tracker):
        if not node:
            return []
        next_paths = dict({goal: [[]]})
        for sum_of_path, paths in tracker.items():
            for path in paths:
                i = sum_of_path - node.val
                if i not in next_paths: #If not already in dict, create new entry
                    next_paths[i] = []
                next_paths[i] += [path + [node.key]] 
        paths = next_paths[0] if 0 in next_paths else [] \
              + _paths_with_sum(node.left, target_sum, next_paths) \
              + _paths_with_sum(node.right, target_sum, next_paths) #Recursively count the number of paths each way
        return paths    
    
    root = binary_tree.root
    partial_paths = dict({target_sum: [[]]})
    return _paths_with_sum(root, target_sum, partial_paths)

**Unit Tests:**

In [17]:
import unittest, math, random

class Test(unittest.TestCase):
    def test_route_between_nodes(self):
        vertices = [0,1,2,3]
        edges = [(0,1), (0, 2), (1, 2), (2, 0), (2, 3), (3, 3)]
        g = build_graph(vertices, edges)
        u, v = g.vertices[1], g.vertices[3]
        self.assertTrue(route_between_nodes(g, u, v))
    def test_minimal_tree(self):
        node_count = 15
        input = MinimalTree([i for i in range(node_count)])
        self.assertTrue(input.max_depth() - 1 <= math.ceil(math.log2(node_count)))
    def test_list_of_depths(self):
        input = BinarySearchTree()
        input.put([8,4,12,2,6,10,14,1,3,7,9])
        result = "8 | 4 => 12 | 2 => 6 => 10 => 14 | 1 => 3 => 7 => 9"
        self.assertEqual(string_builder(list_of_depths(input)), result)
    def test_check_balanced(self):
        input = BinarySearchTree()
        input.put([8,4,12,2,6,10,14,1,3,7,9])
        self.assertTrue(check_balanced(input)) 
        input.root.left.left.left.left = BinaryTreeNode(11) #Make tree unbalanced
        input.root.left.left.left.left.left = BinaryTreeNode(12)
        self.assertFalse(check_balanced(input)) 
    def test_validate_bst(self):
        non_bst = MinimalTree([7,3,1,8,9,11]) #Unsorted 
        self.assertFalse(validate_bst(non_bst))
        bst = MinimalTree([i for i in range(10)]) #Sorted 
        self.assertTrue(validate_bst(bst))
    def test_successor(self):
        input = ParentBinarySearchTree()
        input.put([2,5,1,6,7])
        self.assertEqual(successor(input, input.root.left), input.root)
        self.assertEqual(successor(input, input.root.right.right), input.root.right.right.right)
    def test_build_order(self):
        projects = [1,2,3,4,5]
        dependencies = [ (1,2), (3,4), (1,5), (4,1), (3,2) ]
        self.assertEqual(build_order(projects, dependencies), [3, 4, 1, 5, 2])
    def test_first_common_ancestor(self):
        input = MinimalTree()
        input.put_sorted_array([i for i in range(10)])
        self.assertEqual(first_common_ancestor(input, input.root.left.left, input.root.left.right), input.root.left)
    def test_bst_sequence(self):
        input = BinarySearchTree()
        input.put([2,1,3,4])
        result = [[2, 1, 3, 4], [2, 3, 1, 4], [2, 3, 4, 1]]
        self.assertEqual(bst_sequences(input), result)
    def test_check_subtree(self):
        input_1 = BinarySearchTree()
        input_1.put([1, 2, 3])
        input_2 = BinarySearchTree()
        input_2.put([1,2,3])
        self.assertTrue(check_subtree(input_1, input_2))
        input_3 = BinarySearchTree()
        input_3.put([1,2,5])
        self.assertFalse(check_subtree(input_1, input_3))
    def test_random_node(self):
        m = RandomBinaryTree()
        m.put([1,2,3,4,5])
        self.assertEqual(m.getRandomNode(1).val, 2)
        self.assertEqual(m.getRandomNode(3).val, 4)
    def test_paths_with_sum(self):
        input = KeyBinarySearchTree()
        input.put([("A", 4),("B", 2),("C", 6),("D", 1),("E", 1), ("F", -1)])
        self.assertEqual(paths_with_sum(input, 6), [['A', 'B'], ['C']])
    
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

............
----------------------------------------------------------------------
Ran 12 tests in 0.012s

OK
